# Импорт необходимых библиотек

In [1]:
import geopandas # библиотека для работы с геопространственными данными
import ipyleaflet # библиотека интерактивных виджетов для карты, основанная на ipywidgets
import json # библиотека для работы с JSON-форматом файлов
import os # библиотека функций для работы с операционной системой
import psycopg2 # библиотека для связи с базой данных
import shapely # библиотека, содержащая класы различных геометрических фигур

from contextlib import closing # функция для закрытия соединения с БД при выходе из менеджера контекста
from dotenv import load_dotenv # считывания пар ключ-значение из файла .env и установки их как переменные среды
from ipywidgets.widgets import (Accordion, BoundedFloatText, Button, HBox, HTML, Layout, RadioButtons, Text, 
                                Textarea, ToggleButtons, VBox) # Виджеты для создания интерейса пользователя
from pyproj import Geod # выполнение прямых и обратных геодезических вычислених (длина фигурной геометрии)
from typing import Callable, Optional, Union # для тайп-хинтингов Python

## Игнорируем предупреждения

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Создание интерфейса

## Карта: базовый слой, панель контроля слоёв, полноэкранное отображение

In [3]:
# Создаём объект карты
ukr_map = ipyleaflet.Map(basemap=ipyleaflet.basemaps.OpenStreetMap.Mapnik, 
                         center=(49.0275, 31.482778), zoom=5, max_zoom=20, scroll_wheel_zoom=True)
# Добавляем панель контроля слоёв для возможности скрытия последних
ukr_map.add_control(ipyleaflet.LayersControl(position='topright'))
# Добавляем возможность открытия карты на весь экран
ukr_map.add_control(ipyleaflet.FullScreenControl())
# Добавляем отображение текущего масштаба карты
ukr_map.add_control(ipyleaflet.ScaleControl(position='bottomleft', imperial=False))

## Группы управления: построение маршрута, создание 2D сетки, поле сообщений об ошибках

In [4]:
# Создаём кнопки для построения и удаления маршрута
remove_route_button = Button(description='Remove Route!', button_style='danger')
build_route_button = Button(description='Build Route!', button_style='success')

# Создаём текстовое поле для вывода длины растояния
route_distance_field = Text(placeholder='Route Distance', disabled=True)

# Создаём числовые поля для задания длины и ширины ячейки сетки
cell_width_floatfield = BoundedFloatText(value=15000, min=100.0, max=80000.0, step=100, description='Cell Width (m)')
cell_height_floatfield = BoundedFloatText(value=15000, min=100.0, max=80000.0, step=100, description='Cell Height (m)')

# Создаём кнопки для создания и удаления 2D сетки
create_grid_button = Button(description='Create 2D Grid!', button_style='success')
remove_grid_button = Button(description='Remove 2D Grid!', button_style='danger')

# Создаём поле для отображение ошибок и кнопку для его очистки
errors_textarea = Textarea(placeholder='Error messages will be printed here', 
                           layout=Layout(height='150px', min_height='100px', width='auto'))
clear_errors_textarea_button = Button(description='Clear', button_style='warning')

# Реакция на клик по кнопке 'Clear'
def on_clear_errors_textarea_button_clicked(b) -> None:
    errors_textarea.value = ""
    
clear_errors_textarea_button.on_click(on_clear_errors_textarea_button_clicked)

# Создаём кнопки для выбора слоя, к которому относить маркер
layers_togglebuttons = ToggleButtons(options=['border', 'regions', 'districts'], value='districts',
                                     button_style='info',
                                     tooltips=['Border Layer (only for 2D grid)',
                                               'Regions Layer (for route building and 2D grid creation)',
                                               'Districts Layer (for route building and 2D grid creation)'])

# Настраиваем размещение компонентов каждой из трёх групп
route_controls = VBox([HBox([build_route_button, remove_route_button]), route_distance_field])
grid_controls = VBox([HBox([cell_width_floatfield, create_grid_button]), 
                      HBox([cell_height_floatfield, remove_grid_button])])
errors_controls = VBox([errors_textarea, clear_errors_textarea_button])

# Создаём "гармошку"
accordion = Accordion(children=[route_controls, grid_controls, errors_controls])
accordion.set_title(0, 'Build Route')
accordion.set_title(1, 'Create 2d Grid')
accordion.set_title(2, 'Error Messages')

# Окончательный интерфейс пользователя
application = VBox([ukr_map, layers_togglebuttons, accordion])

# Работа с картой

## Слой "Граница Украины": загрузка данных с БД, создание и добавление слоя

In [5]:
# Функция для создания датафрейма с данными из таблиц БД
def get_geodataframe_data_from_db(sql_query: str) -> geopandas.GeoDataFrame:
    load_dotenv()
    with closing(psycopg2.connect(dbname=os.environ.get('DB_NAME'), 
                                  user=os.environ.get('DB_USER'), 
                                  password=os.environ.get('DB_PASSWORD'))) as conn:
        try:
            return geopandas.read_postgis(sql=sql_query, con=conn)
        except (Exception, psycopg2.DatabaseError) as error:
            accordion.selected_index = 2
            errors_textarea.value = str(error)


# Создаём датафрейм с данными из таблицы regions, в случае успешной операции добавляем слой на карту
if (ukraine_border_data := get_geodataframe_data_from_db('SELECT * FROM border;')) is not None:
    ukraine_border_layer = ipyleaflet.GeoData(geo_dataframe=ukraine_border_data,
                                              style={
                                                  'color': 'purple', 'fillColor': 'pink', 'opacity': 0.5,
                                                  'weight': 1.9, 'dashArray': '2', 'fillOpacity': 0.6
                                              },
                                              hover_style={
                                                  'color': 'pink', 'fillColor': 'purple' , 'fillOpacity': 0.5
                                              },
                                              name='Границы Украины')
    ukr_map.add_layer(ukraine_border_layer)

## Слой "Области Украины": загрузка данных с БД, создание и добавление слоя

In [6]:
# Создаём датафрейм с данными из таблицы regions
ukraine_regions_data = get_geodataframe_data_from_db('SELECT * FROM regions;')
# В случае успешной операции добавляем слой на карту
if ukraine_regions_data is not None:
    ukraine_regions_layer = ipyleaflet.GeoData(geo_dataframe=ukraine_regions_data,
                                               style={
                                                   'color': 'orange', 'fillColor': 'green', 'opacity': 0.5,
                                                   'weight': 1.9, 'dashArray': '2', 'fillOpacity': 0.6
                                                       },
                                               hover_style={
                                                   'color': 'green', 'fillColor': 'orange' , 'fillOpacity': 0.5
                                               },
                                               name='Области Украины')
    ukr_map.add_layer(ukraine_regions_layer)

## Слой "Районы Украины": загрузка данных с БД, создание и добавление слоя

In [7]:
# Создаём датафрейм с данными из таблицы districts
ukraine_districts_data = get_geodataframe_data_from_db('SELECT * FROM districts;')
# В случае успешной операции добавляем слой на карту
if ukraine_districts_data is not None:
    ukraine_districts_layer = ipyleaflet.GeoData(geo_dataframe=ukraine_districts_data,
                                                 style={
                                                     'color': 'blue', 'fillColor': 'yellow', 'opacity': 0.5,
                                                     'weight': 1.9, 'dashArray': '2', 'fillOpacity': 0.6
                                                       },
                                                 hover_style={
                                                     'color': 'yellow', 'fillColor': 'blue' , 'fillOpacity': 0.5
                                                 },
                                                 name='Районы Украины')
    ukr_map.add_layer(ukraine_districts_layer)

## Слой маркеров: добавление/удаление маркеров, кластер маркеров, функции-слушатели

In [8]:
# Создание кластера маркеров, который будет содержать все маркеры, созданные пользователем
user_markers_cluster = ipyleaflet.MarkerCluster(name='Маркеры пользователя')
ukr_map.add_layer(user_markers_cluster)

# Создание шаблона поп-апа
popup_text_template = HTML()


# Функция удаления всех маркеров из кластера маркеров
def clear_marker_cluster(cluster: ipyleaflet.leaflet.MarkerCluster) -> None:
    cluster.markers = ()
    
    
# Функция добавление маркера в кластер маркеров пользователя
def add_marker_to_cluster(marker: ipyleaflet.leaflet.Marker, cluster: ipyleaflet.leaflet.MarkerCluster) -> None:
    current_markers_in_cluster = list(cluster.markers)
    current_markers_in_cluster.append(marker)
    cluster.markers = current_markers_in_cluster

    
# Функция удаления маркера из кластера маркеров пользователя
def remove_marker_from_cluster(marker: ipyleaflet.leaflet.Marker, cluster: ipyleaflet.leaflet.MarkerCluster) -> None:
    current_markers_in_cluster = list(cluster.markers)
    current_markers_in_cluster.remove(marker)
    cluster.markers = current_markers_in_cluster
    
    
# Функция подстановки текста в поп-ап маркера
def set_popup_text(element: ipyleaflet.leaflet.Marker, text: str) -> None:
    popup_text_template.value = text
    element.popup = popup_text_template
    
    
# Функция реакции на перемещение маркера, созданного пользователем
def on_marker_move(marker: ipyleaflet.leaflet.Marker) -> None:
    def callback(*args, **kwargs) -> Callable:
        # Установка текста поп-апа для маркера
        set_popup_text(marker, str(kwargs.get('location')))
    return callback


# Функция реакции на двойной клик по маркеру
def on_marker_double_click(marker: ipyleaflet.leaflet.Marker) -> None:
    def callback(*args, **kwargs) -> Callable:
        # Удаление маркера из кластера
        remove_marker_from_cluster(marker=marker, cluster=user_markers_cluster)
    return callback


# Функция реакции нажатия на карту -> добавление маркера в точке клика
def on_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        # Создаём маркер в точке клика
        default_marker = ipyleaflet.Marker(location=kwargs.get('coordinates'))
        # Добавляем поп-ап с информацией о координатах
        set_popup_text(default_marker, str(kwargs.get('coordinates')))
        # Устанавливаем функцию-прослушку на перемещение маркера
        default_marker.on_move(on_marker_move(default_marker))
        # Устанавливаем функцию-прослушку на перемещение маркера
        default_marker.on_dblclick(on_marker_double_click(default_marker))
        # Добавляем маркер в соответствующий кластер
        add_marker_to_cluster(marker=default_marker, cluster=user_markers_cluster)


# Устанавливаем функцию-прослушку для карты на добавление маркера в точке клика
ukr_map.on_interaction(on_map_click)

## Слой заданного маршрута: генерация запроса, вызов функции БД, вывод на карте

In [9]:
# Создаём переменную маршрута, кластер опорных точек маршрута и иконку для маркеров
route = ipyleaflet.AntPath(name='Маршрут', dash_array=[10, 20], delay=500, color='black', pulse_color='red')
centroids = ipyleaflet.MarkerCluster()
icon = ipyleaflet.Icon(icon_url='./images/red_marker.png', icon_size=[18, 28.8])
# Объединяем маршрут и кластер маркеров в один шар
route_layer_group = ipyleaflet.LayerGroup(layers=(route, centroids), name='Маршрут')
ukr_map.add_layer(route_layer_group)


# Функция генерации парметра функции, который содержит координаты всех маркеров
def get_generated_query() -> str:
    query = 'LINESTRING('
    for marker in user_markers_cluster.markers:
        for coordinate in marker.location[::-1]:
            query += f'{coordinate} '
        query += ','
    query = query.rstrip(' ,')
    query += ')'
    return query


# Обращение к функции БД по постройке маршрута
def get_route_form_db(table_name: str) -> geopandas.GeoDataFrame:
    # load_dotenv()
    with closing(psycopg2.connect(dbname=os.environ.get('DB_NAME'), 
                                  user=os.environ.get('DB_USER'), 
                                  password=os.environ.get('DB_PASSWORD'))) as conn:
        try:
            sql_query = f"SELECT * FROM build_route('{get_generated_query()}', '{table_name}');"
            return geopandas.GeoDataFrame.from_postgis(sql=sql_query, con=conn, geom_col='route_geometry')
        except (Exception, psycopg2.DatabaseError) as error:
            accordion.selected_index = 2
            errors_textarea.value = str(error)


def set_route() -> None:
    route_df = get_route_form_db(layers_togglebuttons.get_interact_value())
    if route_df is not None:
        # Удаляем маркеры пользователя
        clear_marker_cluster(user_markers_cluster)
        # Настраиваем "змейку"
        route.locations = [coords[::-1] for coords in json.loads(route_df.iloc[0, 1])['coordinates']]
        # Настраиваем маркеры в опорных точках маршрута
        if centroids.markers:
            clear_marker_cluster(centroids)
        for coords in route.locations:
            add_marker_to_cluster(ipyleaflet.Marker(location=coords, icon=icon, draggable=False), centroids)  
        # Выводим длину маршрута
        route_distance_field.value = f"{round(Geod(ellps='WGS84').geometry_length(route_df.iloc[0, 0]) / 1000, 3)} km"

        
# Функция реакции на клик по кнопке 'Build Route!'
def on_build_route_button_clicked(b) -> None:
    if user_markers_cluster.markers:
        set_route()

        
# Функция реакции на клик по кнопке 'Remove Route!'
def on_remove_route_button_clicked(b) -> None:
    route.locations = []
    clear_marker_cluster(centroids)
    route_distance_field.value = ''

        
build_route_button.on_click(on_build_route_button_clicked)
remove_route_button.on_click(on_remove_route_button_clicked)

## Слой 2D сетки: генерация запроса, вызов функции БД, вывод на карте

In [10]:
# Создаём переменную слоя 2D сетки
grid_2d_layer = None
numbers_to_layers = {
    'border': ukraine_border_data,
    'regions': ukraine_regions_data,
    'districts': ukraine_districts_data
}


def get_polygon_by_marker_position(tb_name: str) -> shapely.geometry.multipolygon.MultiPolygon:
    m_point = shapely.geometry.Point(user_markers_cluster.markers[0].location[::-1])
    for polygon in numbers_to_layers[tb_name]['geom']:
        if m_point.within(polygon):
            return polygon
    return None


                                 
# Обращение к функции БД по постройке маршрута
def get_grid_from_db(polygon: shapely.geometry.multipolygon.MultiPolygon) -> geopandas.GeoDataFrame:
    # load_dotenv()
    with closing(psycopg2.connect(dbname=os.environ.get('DB_NAME'), 
                                  user=os.environ.get('DB_USER'), 
                                  password=os.environ.get('DB_PASSWORD'))) as conn:
        try:
            sql_query = f"SELECT * FROM create_grid_2d('{str(polygon)}', {int(cell_width_floatfield.value)},\
                          {int(cell_height_floatfield.value)});"
            return geopandas.GeoDataFrame.from_postgis(sql=sql_query, con=conn, geom_col='geom')
        except (Exception, psycopg2.DatabaseError) as error:
            accordion.selected_index = 2
            errors_textarea.value = str(error)


def set_grid_2d() -> None:
    polygon = get_polygon_by_marker_position(layers_togglebuttons.get_interact_value())
    if polygon is not None:
        grid_2d = get_grid_from_db(polygon)
        if grid_2d is not None:
            global grid_2d_layer
            if grid_2d_layer is None:
                grid_2d_layer = ipyleaflet.GeoData(geo_dataframe=grid_2d, 
                                                   style={'color': '#00ffff', 'fillOpacity': 0.2},
                                                   name='2D сетка')
                ukr_map.add_layer(grid_2d_layer)
            else:
                grid_2d_layer.data.clear()
                grid_2d_layer.geo_dataframe = grid_2d
        
              
# Функция реакции на клик по кнопке 'Build Route!'
def on_create_grid_button_clicked(b) -> None:
    if len(user_markers_cluster.markers) == 1:
        set_grid_2d()
    else:
        accordion.selected_index = 2
        errors_textarea.value = 'Для построение 2D сетки необходим один маркер!!!'

        
# Функция реакции на клик по кнопке 'Remove Route!'
def on_remove_grid_button_clicked(b) -> None:
    global grid_2d_layer
    if grid_2d_layer is not None:
        ukr_map.remove_layer(grid_2d_layer)
        grid_2d_layer = None

    
create_grid_button.on_click(on_create_grid_button_clicked)
remove_grid_button.on_click(on_remove_grid_button_clicked)

In [11]:
application